This Notebook launches a study comparing the different variants of numerical fluxes for a Mach 4 supersonic flow over a blunt body presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy }

Get Default Queue

In [3]:
BoSSSshell.ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client Default @DC3, @\\dc3\userspace\sebastian\cluster\binaries"
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client Default @DC3, @\\dc3\userspace\sebastian\cluster\binaries"
2,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client @C:\Users\sebastian\AppData\Local\BoSSS-LocalJobs


default database path

In [4]:
BoSSSshell.WorkflowMgm.DefaultDatabase.Path

\\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy

list of available queues

In [5]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


Here we prescribe all combinations we want to study

In [6]:
using XESF.Fluxes;
using ApplicationWithIDT;

int[] DGdegrees = new int[] {2};
int[] Grid_Res = new int[] {2};
double[] Aggs = new double[] {0.4};
var iFluxes = new int[] {0,1,2,3}; //{ ConvectiveInterfaceFluxes.GodunovInterface,ConvectiveInterfaceFluxes.RoeInterface, ConvectiveInterfaceFluxes.CentralFluxInterface, ConvectiveInterfaceFluxes.OptimizedHLLCInterface };
var wallFluxes=new int[] {0}; //{ ConvectiveInterfaceFluxes.OptimizedHLLCWall, ConvectiveInterfaceFluxes.RoeWall};
var bulkFluxes=new int[] {0}; // { ConvectiveBulkFluxes.OptimizedHLLC, ConvectiveBulkFluxes.Roe, ConvectiveBulkFluxes.CentralFlux, ConvectiveBulkFluxes.Godunov};
var iProbs= new int[] {0}; //{ OptProblemType.FullEnRes, OptProblemType.EnResOnlyNearBand, OptProblemType.RankineHugoniot};
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;


get data we need to initialize the initial guess for the shock LevelSet

In [7]:
var bowShockPoints =new Tuple<byte[],string>( System.IO.File.ReadAllBytes("BowShockPoints.txt"),"BowShockPoints.txt");
var LevelSetDB =new Tuple<byte[],string>(System.IO.File.ReadAllBytes("bosss_db_levelSets.zip"),"bosss_db_levelSets.zip");

run the study

In [8]:
foreach(double iProb in iProbs) {
foreach(double iflux in iFluxes) {
    foreach(double wflux in wallFluxes) {
        foreach(double bflux in bulkFluxes) {
foreach(double agg in Aggs) {
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                Job j = new Job(string.Format($"FS-XDGBS-p{p}-{5*n}x{16*n}-agg{agg}-iPrb{iProb}-iFlx{iflux}-wFLx{wflux}-bFlx{bflux}-Fphi{0}"), typeof(XESF.XESFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESF.XESFHardCodedControl.XDGBS_Cluster(DegE:{p},numX:{5*n},numY:{16*n},agg:{agg},plotInterval:{-1},iProb:{iProb},iflux:{iflux},bulkflux:{bflux},wallflux:{wflux},aRI:true,dbPath:@\"{dbPath}\")");
                j.AdditionalDeploymentFiles.Add(bowShockPoints);
                j.AdditionalDeploymentFiles.Add(LevelSetDB);
                j.NumberOfThreads=1;
                j.Activate(BoSSSshell.ExecutionQueues[0]); 
        };                                                                                                                                                          
    };
    }
        }
        }
};
}


Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job FS-XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\sebastian\cluster\binaries\XESF_BowShock_FluxStudy-XESF2024Apr03_213538.532291
copied 50 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job FS-XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\sebastian\cluster\binaries\XESF_BowShock_FluxStudy-XESF2024Apr03_213550.956164
copied 50 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job FS-XDGBS-p2-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\sebastian\cluster\binaries\XESF_BowShock_FluxStudy-XESF2024Apr03_213603.321863
copied 50 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job FS-XDGBS-p2-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\sebastian\cluster\binaries\XESF_BowShock_FluxStudy-XESF2024Apr03_213616.138528
copied 50 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

